## Case Study 1 DS7333 

## Business Understanding

The objective behind this case study is to build a linear regression modeling using L1 (LASSO) or L2 (Ridge) regularization to predict the cirtical temperature.  The team was given two files which contain the data and from this data we must attempt to predict the cirtical temperature.  

Our overall goals are to predict critical temperature and to describe which variable carries the most importance.

## Data Evaluation/ Engineering 

In [1]:
#First we'll import all our necessary libraries and add addtional ones as needed as the project grows
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from ml_metrics import rmse
import matplotlib.pyplot as plt
from sklearn import datasets
from pandas_profiling import ProfileReport
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn import preprocessing
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from matplotlib import pyplot

In [2]:
#Next we'll bring in our data
dat1 = pd.read_csv("unique_m.csv")
dat2 = pd.read_csv("train.csv")

#Drop critical temp
dat1.drop(['critical_temp'], axis=1)

#Merge the data
frames = [dat1,dat2]
result = pd.concat(frames)


In [3]:
#Drop material
del result['material']

In [ ]:
result.info()

In [ ]:
result.describe()

In [4]:
pd.set_option('display.max_rows', None)
#Check out the data, start to make some decisions on columns and missing data
#Compute percentages of each columns missing data
percent_missing = result.isnull().sum()/len(result)
#Put percents into dfReduced
missing_value_result = pd.DataFrame({'column_name': result.columns,
'percent_missing': percent_missing})
#Sort it and show the results
missing_value_result.sort_values('percent_missing', inplace=True)
missing_value_result.round(6)

,column_name,percent_missing
critical_temp,critical_temp,0.0
H,H,0.5
mean_atomic_radius,mean_atomic_radius,0.5
wtd_mean_atomic_radius,wtd_mean_atomic_radius,0.5
gmean_atomic_radius,gmean_atomic_radius,0.5
wtd_gmean_atomic_radius,wtd_gmean_atomic_radius,0.5
entropy_atomic_radius,entropy_atomic_radius,0.5
wtd_entropy_atomic_radius,wtd_entropy_atomic_radius,0.5
range_atomic_radius,range_atomic_radius,0.5
wtd_range_atomic_radius,wtd_range_atomic_radius,0.5


Based on the analysis above, we are lucky and have no missing values to impute/handle.  Additionally, we will check for NaN's in the combined dataframe.

In [5]:
result.isnull().values.any()


True

In [6]:
result= result.fillna(0)

This line handles filling the NaN's with 0's.

In [7]:
print(len(result))
result.drop_duplicates(keep = False, inplace = True)
print(len(result))

42526
42395


In a seperate check for duplicate entries- we can see that we have no duplicates.

In the cell below we will examine the data for multicollinearity, and remove variables above a threshold of 1000.  

In [ ]:
vif_info = pd.DataFrame()
vif_info['VIF'] = [variance_inflation_factor(result.values, i) for i in range(result.shape[1])]
vif_info['Column'] = result.columns
vif_info.sort_values('VIF', ascending=False)

In [9]:
result.describe()

,H,He,Li,Be,B,C,N,O,F,Ne,...,wtd_gmean_ThermalConductivity,entropy_ThermalConductivity,wtd_entropy_ThermalConductivity,wtd_range_ThermalConductivity,wtd_std_ThermalConductivity,wtd_mean_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence
count,42395.000000,42395.0,42395.000000,42395.000000,42395.000000,42395.000000,42395.000000,42395.000000,42395.00000,42395.0,...,42395.000000,42395.000000,42395.000000,42395.000000,42395.000000,42395.000000,42395.000000,42395.000000,42395.000000,42395.000000
mean,0.008870,0.0,0.006081,0.017372,0.071518,0.193079,0.006663,1.509214,0.00746,0.0,...,13.537408,0.362756,0.268920,31.005491,48.151207,1.567453,1.018894,0.737131,0.418728,0.336138
std,0.189449,0.0,0.091947,0.601178,0.743123,3.127654,0.106738,3.090365,0.09386,0.0,...,31.476891,0.430716,0.351216,43.534451,65.986942,1.782205,1.347629,1.011168,0.542056,0.466432
min,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.0,...,6.084459,0.738694,0.541858,56.556240,114.697446,2.590909,2.000000,1.061829,0.800000,0.499857
max,14.000000,0.0,3.000000,40.000000,105.000000,120.000000,12.800000,66.000000,4.00000,0.0,...,376.032878,1.633977,1.612989,401.440000,213.300452,7.000000,6.000000,6.992200,3.000000,3.000000


In [8]:
del result['wtd_mean_fie']
del result['wtd_gmean_fie']
del result['mean_fie']
del result['gmean_fie']
del result['wtd_mean_atomic_radius']
del result['wtd_gmean_atomic_radius']
del result['mean_atomic_radius']
del result['entropy_fie']
del result['gmean_atomic_radius']
del result['wtd_gmean_Valence']
del result['mean_Valence']
del result['gmean_Valence']
del result['entropy_Valence']
del result['wtd_gmean_atomic_mass']
del result['mean_atomic_mass']
del result['wtd_entropy_atomic_radius']
del result['gmean_atomic_mass']
del result['wtd_entropy_Valence']
del result['entropy_atomic_mass']
del result['std_atomic_radius']
del result['std_fie']
del result['wtd_std_fie']
del result['wtd_std_atomic_radius']
del result['wtd_mean_ElectronAffinity']
del result['std_ThermalConductivity']
del result['wtd_entropy_fie']
del result['range_fie']
del result['wtd_entropy_atomic_mass']
del result['range_ThermalConductivity']
del result['entropy_Density']
del result['wtd_mean_FusionHeat']

Based on the results above- this data is ready for analysis.  It is free of duplicates, NaN's, and missing values. 


With those items stated- we should also discuss that we are soely interested in our outcome variable 

### EDA

The purpose of the plot above was to investigate the distribution of values of our outcome variable.  Based on this histogram we can see that the data is heavily right skewed.  This may be worth log-transforming to fit our normality assumption.

## Modeling Preparations

The following evaluation metrics will be used for the regression task. -Mean Absolute Error (MAE) -Root Mean Squared Error (RMSE) -Mean Absolute Percentage Error (MAPE) and R^2

Mean absolute error is being used because it is an intuitive metric that simply looks at the absolute difference between the data and the models preditions. This metric is a good first pass at evaluating a models performance, its simple and quick. The mean absolute error however, does not indicate under/over performance in the model. The residuals all contribute proportionally, so larger errors will contribute significantally to the model. A small MAE is a good indicator of good prediction and a large MAE indicates the model may need more work. MAE of 0 is ideal, but not really possible. The MAE is farily robust to outliers since it uses the absolute value. For this model a lower MAE is "better." This metric is appropriate since we are concerned with the models ability to predict critical temperatures.

Root mean squared error is the standard deviation of the residuals (commonly known as prediction errrors). We use these rediuals to see how far from the line the points are. RMSE is a good metric to tell us how concentrated the data is around the line of best fit. The nice and differentiating part of RMSE is that they can be positive or negative as the predicted value under or over the estimates of the actual value unlike the MAE and MAPE which use the absolute value. Additionally we can use the RMSE as a good measure of the spread of the y values about the predicted y values.

Mean absolute percentage error is being used as the percentage equivalent of MAE. Similarily to the MAE, MAPE measures how far our model's predictions are off from their corresponding outputs on average. MAPE is pretty easy to interpret because of its use of percentages. As a downside, MAPE is undefined for data where we have 0 values. The MAPE can also grow very large if the values themselves are small, and as a final note MAPE is biased towards predictions that are systematically less than actual values. We will use MAPE as a more interpretable MAE.
We will interpret the MAPE in terms of percentages, as an example the MAPE will state, "our model's predictions are on average xx% off from the actual value." This metric is appropriate since we are concerned with the model's ability to predict critical temperatures, furthermore the addition of percentage will further our ability to interpret the model's performance.

Finally R2 or the coefficient of determination will be used and is the proportion of the variation in the dependent variable that is predictable from the independent variable.  This isn't the best metric, but it is quite easy to interpret as it lives in a 0-100% scale.  


## Model Building & Evaluations

### Start with Linear Regression Model

In [ ]:
# Lets scale the data
x = result.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
result = pd.DataFrame(x_scaled)

In [ ]:
# Create training and testing sets (cross-validation not needed)
train_set = result.sample(frac=0.8, random_state=100)
test_set = result.loc[:, result.columns != 'critical_temp']
print(train_set.shape[0])
print(test_set.shape[0])


In [ ]:
# Get the training and testing row indices for later use
train_index = train_set.index.values.astype(int)
test_index = test_set.index.values.astype(int)

In [ ]:
# Converting the training and testing datasets back to matrix-formats
X_train = train_set.iloc[:, :].values # returns the data; excluding the target
Y_train = train_set.iloc[:, -1].values # returns the target-only
X_test = test_set.iloc[:, :].values # ""
Y_test = test_set.iloc[:, -1].values # "

In [ ]:
# Fit a linear regression to the training data
reg = LinearRegression(normalize=True).fit(X_train, Y_train)
print(reg.score(X_train, Y_train))
print(reg.coef_)
print(reg.intercept_)
print(reg.get_params())

In [ ]:
# Find the variable with the largest "normalized" coefficient value
print('The positive(max) coef-value is {}'.format(max(reg.coef_))) # Positive Max
#print('The abs(max) coef-value is {}'.format(max(reg.coef_, key=abs))) # ABS Max
max_var = max(reg.coef_) # Positive Max
#max_var = max(reg.coef_, key=abs) # ABS Max
var_index = reg.coef_.tolist().index(max_var)
print('The variable associated with this coef-value is {}'.format(result.columns[var_index]))

In [ ]:
importance = reg.coef_
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

In [ ]:
Y_pred = reg.predict(X_test)

orig_mae = mean_absolute_error(Y_test,Y_pred)
orig_mse = mean_squared_error(Y_test,Y_pred)
orig_rmse_val = rmse(Y_test,Y_pred)
orig_r2 = r2_score(Y_test,Y_pred)
print("MAE: %.30f"%orig_mae)
print("MSE:  %.30f"%orig_mse)
print("RMSE:  %.30f"%orig_rmse_val)
print("R2:  %.30f"%orig_r2)

In [ ]:
Y_pred_tr = reg.predict(X_train)

orig_mae_tr = mean_absolute_error(Y_train,Y_pred_tr)
orig_mse_tr = mean_squared_error(Y_train,Y_pred_tr)
orig_rmse_val_tr = rmse(Y_train,Y_pred_tr)
orig_r2_tr = r2_score(Y_train,Y_pred_tr)
print("MAE: %.30f"%orig_mae_tr)
print("MSE:  %.30f"%orig_mse_tr)
print("RMSE:  %.30f"%orig_rmse_val_tr)
print("R2:  %.30f"%orig_r2_tr)

In [ ]:
res_frame = pd.DataFrame({'data':'original',
                   'imputation':'none',
                   'mae': orig_mae, 
                   'mse': orig_mse, 
                   'rmse':orig_rmse_val, 
                   'R2':orig_r2,
                   'mae_diff':np.nan,
                   'mse_diff':np.nan,
                   'rmse_diff':np.nan,
                   'R2_diff':np.nan}, index=[0])

In [ ]:
res_frame

## LASSO Regularization 

In [ ]:
l1_mod = linear_model.Lasso(alpha=0.001, normalize=True).fit(X_train, Y_train)
print(l1_mod.score(X_train, Y_train))
print(l1_mod.coef_)
print(l1_mod.intercept_)
print(l1_mod.get_params())

In [ ]:
Y_pred2 = l1_mod.predict(X_test)

orig_mae2 = mean_absolute_error(Y_test,Y_pred2)
orig_mse2 = mean_squared_error(Y_test,Y_pred2)
orig_rmse_val2 = rmse(Y_test,Y_pred2)
orig_r22 = r2_score(Y_test,Y_pred2)
print("MAE: %.5f"%orig_mae2)
print("MSE:  %.5f"%orig_mse2)
print("RMSE:  %.5f"%orig_rmse_val2)
print("R2:  %.5f"%orig_r22)

In [ ]:
Y_pred2_tr = l1_mod.predict(X_train)

orig_mae2_tr = mean_absolute_error(Y_train,Y_pred2_tr)
orig_mse2_tr = mean_squared_error(Y_train,Y_pred2_tr)
orig_rmse_val2_tr = rmse(Y_train,Y_pred2_tr)
orig_r22_tr = r2_score(Y_train,Y_pred2_tr)
print("MAE: %.5f"%orig_mae2_tr)
print("MSE:  %.5f"%orig_mse2_tr)
print("RMSE:  %.5f"%orig_rmse_val2_tr)
print("R2:  %.5f"%orig_r22_tr)

In [ ]:
# Find the variable with the largest "normalized" coefficient value
print('The positive(max) coef-value is {}'.format(max(l1_mod.coef_))) # Positive Max
#print('The abs(max) coef-value is {}'.format(max(l1_mod.coef_, key=abs))) # ABS Max
max_var = max(l1_mod.coef_) # Positive Max
#max_var = max(reg.coef_, key=abs) # ABS Max
var_index = l1_mod.coef_.tolist().index(max_var)
print('The variable associated with this coef-value is {}'.format(result.columns[var_index]))

In [ ]:
importance = l1_mod.coef_
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

In [ ]:
res_frame2 = pd.DataFrame({'data':'lasso',
                   'imputation':'none',
                   'mae': orig_mae2, 
                   'mse': orig_mse2, 
                   'rmse':orig_rmse_val2, 
                   'R2':orig_r22,
                   'mae_diff':orig_mae2-orig_mae,
                   'mse_diff':orig_mse2-orig_mse,
                   'rmse_diff':orig_rmse_val2-orig_rmse_val,
                   'R2_diff':orig_r22-orig_r2}, index=[0])

In [ ]:
res_frame = pd.concat([res_frame, res_frame2])
res_frame

## Ridge Regularization

In [ ]:
l2_mod = Ridge(alpha=1.0, normalize=True).fit(X_train, Y_train)
print(l2_mod.score(X_train, Y_train))
print(l2_mod.coef_)
print(l2_mod.intercept_)
print(l2_mod.get_params())

In [ ]:
Y_pred3 = l2_mod.predict(X_test)

orig_mae3 = mean_absolute_error(Y_test,Y_pred3)
orig_mse3 = mean_squared_error(Y_test,Y_pred3)
orig_rmse_val3 = rmse(Y_test,Y_pred3)
orig_r23 = r2_score(Y_test,Y_pred3)
print("MAE: %.5f"%orig_mae3)
print("MSE:  %.5f"%orig_mse3)
print("RMSE:  %.5f"%orig_rmse_val3)
print("R2:  %.5f"%orig_r23)

In [ ]:
Y_pred3_tr = l2_mod.predict(X_train)

orig_mae3_tr = mean_absolute_error(Y_train,Y_pred3_tr)
orig_mse3_tr = mean_squared_error(Y_train,Y_pred3_tr)
orig_rmse_val3_tr = rmse(Y_train,Y_pred3_tr)
orig_r23_tr = r2_score(Y_train,Y_pred3_tr)
print("MAE: %.5f"%orig_mae3_tr)
print("MSE:  %.5f"%orig_mse3_tr)
print("RMSE:  %.5f"%orig_rmse_val3_tr)
print("R2:  %.5f"%orig_r23_tr)

In [ ]:
# Find the variable with the largest "normalized" coefficient value
print('The positive(max) coef-value is {}'.format(max(l2_mod.coef_))) # Positive Max
print('The abs(max) coef-value is {}'.format(max(l2_mod.coef_, key=abs))) # ABS Max
max_var = max(l2_mod.coef_) # Positive Max
#max_var = max(reg.coef_, key=abs) # ABS Max
var_index = l2_mod.coef_.tolist().index(max_var)
print('The variable associated with this coef-value is {}'.format(result.columns[4]))

In [ ]:
importance = l2_mod.coef_
# summarize feature importance
for i,v in enumerate(importance):
	print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()

In [ ]:
res_frame3 = pd.DataFrame({'data':'ridge',
                   'imputation':'none',
                   'mae': orig_mae3, 
                   'mse': orig_mse3, 
                   'rmse':orig_rmse_val3, 
                   'R2':orig_r23,
                   'mae_diff':orig_mae3-orig_mae,
                   'mse_diff':orig_mse3-orig_mse,
                   'rmse_diff':orig_rmse_val3-orig_rmse_val,
                   'R2_diff':orig_r23-orig_r2}, index=[0])

In [ ]:
res_frame = pd.concat([res_frame, res_frame3])
res_frame

In [ ]:
result.describe()

## Model Interpretability & Explainability

## Case Conclusions